`63070501061 S.RAKNA`

> 30 points, 2 hrs.

# CNN

- Modify the `CNN 10 Classes` program shown in class to make it classify the 7 people’s faces used in the `Eigenfaces` program. 
- Here you should not reduce the number of dimensions, but use the original image before PCA. 
- You can look into this site https://duchesnay.github.io/pystatsml/auto_gallery/ml_lab_face_recognition.html under ***Basic ConvNet*** to help you decide what architecture to choose for your CNN. 
- Report your precision and macro average F-1 score

In [ ]:
import tarfile

import matplotlib.pyplot as plt
import numpy as np
import pylab as pl
import tensorflow as tf
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

In [ ]:
# Download and prepare the Eigenfaces dataset
tfile = tarfile.open("../eigenfaces/lfw-funneled.tgz", "r:gz")
tfile.extractall(".")

In [ ]:
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

In [ ]:
n_samples, h, w = lfw_people.images.shape  # type: ignore
np.random.seed(42)


In [ ]:
n_samples, h, w

In [ ]:
X = lfw_people.data # type: ignore
n_features = X.shape[1]

In [ ]:
y = lfw_people.target # type: ignore
target_names = lfw_people.target_names # type: ignore
n_classes = target_names.shape[0]
print("target y = \n", y)  # target 0-6 for 7 people
print("target names = \n", target_names)


In [ ]:
print("Total dataset size:")
print("n_samples: %d" % n_samples)
print("n_features: %d" % n_features)
print("n_classes: %d" % n_classes)

In [ ]:
# Split into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

In [ ]:
def plot_gallery(images, titles, h, w, n_row=3, n_col=4):
    """Helper function to plot a gallery of portraits"""
    pl.figure(figsize=(1.8 * n_col, 2.4 * n_row))
    pl.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
    for i in range(n_row * n_col):
        pl.subplot(n_row, n_col, i + 1)
        pl.imshow(images[i].reshape((h, w)), cmap=pl.cm.gray) # type: ignore
        pl.title(titles[i], size=12)
        pl.xticks(())
        pl.yticks(())

In [ ]:
plot_gallery(X_train, target_names[y_train], h, w, 5, 5)

### CNN Section

In [ ]:
input_shape = (h, w, 1)

model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation="relu", input_shape=input_shape))
model.add(layers.Conv2D(32, (3, 3), activation="relu"))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation="relu"))
model.add(layers.Conv2D(64, (3, 3), activation="relu"))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation="relu"))
model.add(layers.Conv2D(128, (3, 3), activation="relu"))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dense(128, activation="relu"))
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(n_classes, activation="softmax"))

model.summary()


In [ ]:
# compile the model
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
loss = tf.keras.losses.CategoricalCrossentropy()

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=["accuracy"]
)

history = model.fit(
    X_train.reshape(-1, h, w, 1),
    tf.keras.utils.to_categorical(y_train),
    epochs=30,
    validation_data=(X_test.reshape(-1, h, w, 1), tf.keras.utils.to_categorical(y_test)),
)


In [ ]:
# Evaluate the model and plot training and validation loss and accuracy
test_loss, test_acc = model.evaluate(
    X_test.reshape(-1, h, w, 1),
    tf.keras.utils.to_categorical(y_test),
    verbose=2
)
print("test loss = ", test_loss)
print("test accuracy = ", test_acc)

# plot the training and validation accuracy
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()